In [24]:
import os
import openai
from git import Repo
from pathlib import Path
import shutil
from bs4 import BeautifulSoup as Soup

In [3]:
pwd

'/Users/orennisim/Desktop/Udemy Courses/OpenAI python API bootcamp/Exercies/Automatic Blog post'

In [5]:
PATH_TO_BLOG_REPO = Path('/Users/orennisim/Desktop/Udemy Courses/OpenAI python API bootcamp/Exercies/Automatic Blog post/.git')

In [6]:
PATH_TO_BLOG_REPO.parent

PosixPath('/Users/orennisim/Desktop/Udemy Courses/OpenAI python API bootcamp/Exercies/Automatic Blog post')

In [7]:
PATH_TO_BLOG = PATH_TO_BLOG_REPO.parent

In [8]:
PATH_TO_CONTENT = PATH_TO_BLOG/"content"

In [9]:
PATH_TO_CONTENT

PosixPath('/Users/orennisim/Desktop/Udemy Courses/OpenAI python API bootcamp/Exercies/Automatic Blog post/content')

In [11]:
PATH_TO_CONTENT.mkdir(exist_ok=True,parents=True)

In [12]:
#Function that updates our blogs via commiting to the Repository.
def update_blog(commit_message="Updated blog"):
    #GitPython --Repo Location
    repo = Repo(PATH_TO_BLOG_REPO)
    #git add .
    repo.git.add(all=True)
    #git commit .
    repo.index.commit(commit_message)
    # git push
    origin = repo.remote(name='origin')
    origin.push()

In [14]:
def create_new_blog(title, content, cover_image=Path("../PT Centered Purple.png")):
    
    cover_image = Path(cover_image)
    
    #count the number of html files that we have
    files =len(list(PATH_TO_CONTENT.glob("*.html")))
    new_title = f"{files+1}.html"
    path_to_new_content = PATH_TO_CONTENT/new_title
    
    shutil.copy(cover_image, PATH_TO_CONTENT)
    if not os.path.exists(path_to_new_content):
        # Write a new HTML file
        with open(path_to_new_content, "w") as f:
            
            #The head of the HTML page
            f.write("<!DOCTYPE html>\n")
            f.write("<html>\n")
            f.write("<head>\n")
            f.write(f"<title> {title} </title>\n")
            f.write("</head>\n")
            
            f.write("<body>\n")
            
            #Insert the image from DALEE
            f.write(f"<img src='{cover_image.name}' alt='Cover Image'> <br />\n")
            f.write(f"<h1> {title} </h1>")
            
            #Insert the recipe from OpenAI
            f.write(content.replace("\n", "<br />\n"))
            f.write("</body>\n")
            f.write("</html>\n")
            print("Blog created")
            return path_to_new_content
    else:
        raise FileExistsError("File already exist! Abort")

In [15]:
path_to_new_content = create_new_blog('Test Title', 'holla amogissf fdhfjsn', 'Bacon and Egg French Toast.png')

Blog created


In [31]:
# Index.html --> Blog posts
with open(PATH_TO_BLOG/"index.html") as index:
    soup = Soup(index.read())


In [33]:
soup

<html lang="en">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<title>Blog</title>
</head>
<body>
<h1>Hello get ready for some Recipes!!</h1>
<a href="index.
    ">Home</a>
</body>
</html>

In [32]:
def check_for_duplicate_links(path_to_new_content, links):
    #A list of current existing blogs
    urls = [str(link.get("href")) for link in links]
    content_path = str(Path(*path_to_new_content.parts[-2:]))
    return content_path in urls
    

In [ ]:
def write_to_index(path_to_new_content):
    with open(PATH_TO_BLOG/"index.html") as index:
        soup = Soup(index.read())

    links = soup.find_all("a")
    last_link = links[-1]
    
    if check_for_duplicate_links(path_to_new_content, links):
        raise ValueError("Link does already exist!")
        
    link_to_new_blog = soup.new_tag("a", href=Path(*path_to_new_content.parts[-2:]))
    link_to_new_blog.string = path_to_new_content.name.split(".")[0]
    last_link.insert_after(link_to_new_blog)
    
    with open(PATH_TO_BLOG/"index.html", "w") as f:
        f.write(str(soup.prettify(formatter='html')))